# Instructions
## Data preprocessing before loading to Python
1. When I load jobtittles1 file, I noticed that the uncode format is incorrect. Therefore I save the file with correct format and name it as jobtitles1.csv.
2. There is a missing "," in department.jsons file. So I added back to the departments.json and save it. 

In [9]:
import json 
import pandas as pd

with open("departments.json") as dep_data:
    dep = dep_data.read()

train_d = json.loads(dep)   


In [101]:
test=pd.DataFrame()

for i in range(len(train_d)): 
    for key,value in train_d[i].items() :
        for x in value:
            test = test.append({'title': x, 'department':key}, ignore_index=True)

test.head()

department          title
0        Defense        Defence
1        Defense          Space
2        Defense       Military
3  Entertainment      Animation
4  Entertainment  Entertainment

In [102]:
train=pd.DataFrame()

for i in range(len(train_d)): 
    for key,value in train_d[i].items() :            
        train = train.append({'title':' '.join(value), 'department':key}, ignore_index=True)

train['tokenized'] = train.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)
test['tokenized'] = test.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

In [112]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import *

train['tokenized'] = [[w.lower() for w in row] for row in train['tokenized']]
train['tokenized'] = [[w for w in row if re.search('^[a-z]+$',w)] for row in train['tokenized']]
    
stop_list = stopwords.words('english')
train['tokenized'] = [[w for w in row if w not in stop_list] for row in train['tokenized']]
    
stemmer = PorterStemmer()
train['tokenized'] = [[stemmer.stem(w) for w in row] for row in train['tokenized']]

test['tokenized'] = [[w.lower() for w in row] for row in test['tokenized']]
test['tokenized'] = [[w for w in row if re.search('^[a-z]+$',w)] for row in test['tokenized']]
    
stop_list = stopwords.words('english')
test['tokenized'] = [[w for w in row if w not in stop_list] for row in test['tokenized']]
    
stemmer = PorterStemmer()
test['tokenized'] = [[stemmer.stem(w) for w in row] for row in test['tokenized']]


In [124]:
preprocess(train['tokenized'])
preprocess(test['tokenized'])

In [123]:
defense_dictionary = corpora.Dictionary(train['tokenized'][0:1])
print(defense_dictionary)


Dictionary(3 unique tokens: ['defenc', 'militari', 'space'])


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [119]:
query1 = [stemmer.stem('Defence'), stemmer.stem('space')]
query1_vec = defense_dictionary.doc2bow(query1)
print(query1_vec)

sg_tfidf = models.TfidfModel(sg_vecs)
query1_vec_tfidf = sg_tfidf[query1_vec]
print(query1_vec_tfidf)


[(0, 1), (2, 1)]


NameError: name 'sg_tfidf' is not defined

In [ ]:
from gensim import models
amazon_tfidf = models.TfidfModel(amazon_vecs)
amazon_vecs_with_tfidf = [amazon_tfidf[vec] for vec in amazon_vecs]


In [ ]:
from gensim import similarities
amazon_index = similarities.SparseMatrixSimilarity(amazon_vecs_with_tfidf, 16353)